# NLP Introdução a regex e modelos de linguagem

## Objetivos:
* Avance nos estudos sobre Processamento de Linguagem Natural.
* Aprenda como as regex pode ajudar no tratamento de dados textuais.
* Entenda o que são o modelos de linguagem e suas aplicações
* Crie um modelo que detecta idiomas modo automatizado.
* Pratique bibliotecas python como NLTK e Scikit-Learn.

link: https://cursos.alura.com.br/course/nlp-modelos-linguagem

## Importando as bibliotecas necessárias

In [57]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.util import bigrams
from nltk.lm import MLE, Laplace
from nltk.lm.preprocessing import pad_both_ends, padded_everygram_pipeline
from sklearn.model_selection import train_test_split

## Carregando os dados

In [2]:
url_portugues = 'https://raw.githubusercontent.com/alura-cursos/nlp-modelos-linguagem/master/dataset/stackoverflow_portugues.csv'
url_espanhol = 'https://raw.githubusercontent.com/alura-cursos/nlp-modelos-linguagem/master/dataset/stackoverflow_espanhol.csv'
url_ingles = 'https://raw.githubusercontent.com/alura-cursos/nlp-modelos-linguagem/master/dataset/stackoverflow_ingles.csv'

df_portugues = pd.read_csv(url_portugues)
df_espanhol = pd.read_csv(url_espanhol)
df_ingles = pd.read_csv(url_ingles)

df_portugues.head()

,Id,Título,Questão,Tags,Pontuação,Visualizações
0,2402,Como fazer hash de senhas de forma segura?,"<p>Se eu fizer o <em><a href=""http://pt.wikipe...",<hash><segurança><senhas><criptografia>,350,22367
1,6441,Qual é a diferença entre INNER JOIN e OUTER JOIN?,<p>Qual é a diferença entre <code>INNER JOIN</...,<sql><join>,276,176953
2,579,Por que não devemos usar funções do tipo mysql_*?,<p>Uma dúvida muito comum é por que devemos pa...,<php><mysql>,226,9761
3,2539,As mensagens de erro devem se desculpar?,<p>É comum encontrar uma mensagem de erro que ...,<aplicação-web><gui><console><ux>,214,5075
4,17501,"Qual é a diferença de API, biblioteca e Framew...",<p>Me parecem termos muito próximos e eventual...,<api><framework><terminologia><biblioteca>,193,54191


In [3]:
print(f'Os dados em português possuem {df_portugues.shape[0]} observações e {df_portugues.shape[1]} atributos')
print(f'Os dados em ingles possuem {df_ingles.shape[0]} observações e {df_ingles.shape[1]} atributos')
print(f'Os dados em espanhol possuem {df_espanhol.shape[0]} observações e {df_espanhol.shape[1]} atributos')

Os dados em português possuem 500 observações e 6 atributos
Os dados em ingles possuem 500 observações e 6 atributos
Os dados em espanhol possuem 500 observações e 6 atributos


In [4]:
questao_espanhol = df_espanhol.Questão[5]
print(questao_espanhol)

<p>Siempre he visto que en <code>JavaScript</code> hay:</p>

<ul>
<li>asignaciones <code>=</code></li>
<li>comparaciones <code>==</code> y <code>===</code></li>
</ul>

<p>Creo entender que <code>==</code> hace algo parecido a comparar el valor de la variable y el <code>===</code> también compara el tipo (como un equals de java). </p>

<p><strong>¿Alguien podría confirmarme este punto y extenderlo?</strong>. Soy javero y el no tipado de javascript a veces me encanta y otras lo odio.</p>

<hr>

<p>¿Cuál es la manera correcta en javascript de comparar <code>undefined</code>, <code>null</code> y otros valores por defecto? </p>

<pre><code>variable == null
variable === null
</code></pre>

<p>¿<strong><code>undefined</code></strong> se usa como cadena de texto o como palabra clave? ¿Cual de las siguientes comparaciones es la correcta para un elemento <code>html</code> sin <code>value</code>? (por ejemplo un label sin contenido)</p>

<pre><code>variable == "undefined"
variable === "undefined"

In [5]:
questao_ingles = df_ingles.Questão[5]
print(questao_ingles)

<p>What is the use of the <code>yield</code> keyword in Python? What does it do?</p>

<p>For example, I'm trying to understand this code<sup><strong>1</strong></sup>:</p>

<pre><code>def _get_child_candidates(self, distance, min_dist, max_dist):
    if self._leftchild and distance - max_dist &lt; self._median:
        yield self._leftchild
    if self._rightchild and distance + max_dist &gt;= self._median:
        yield self._rightchild  
</code></pre>

<p>And this is the caller:</p>

<pre><code>result, candidates = [], [self]
while candidates:
    node = candidates.pop()
    distance = node._get_dist(obj)
    if distance &lt;= max_dist and distance &gt;= min_dist:
        result.extend(node._values)
    candidates.extend(node._get_child_candidates(distance, min_dist, max_dist))
return result
</code></pre>

<p>What happens when the method <code>_get_child_candidates</code> is called?
Is a list returned? A single element? Is it called again? When will subsequent calls stop?</p>

<hr>

<

In [6]:
questao_portugues = df_portugues.Questão[5]
print(questao_portugues)

<p>Desenvolvi uma página em PHP para uso interno da empresa que trabalho e apenas pouquíssimas pessoas a utilizam. Através dessa página é possível fazer algumas consultas, inserções, alterações e remoções de dados de uma tabela em um banco de dados MySQL, porém eu acredito que meu código em PHP não está protegido contra injeção de código SQL, por exemplo:</p>

<pre><code>//----CONSULTA SQL----//
$busca = mysql_query ('insert into Produtos (coluna) values(' . $valor . ')');
</code></pre>

<p>Logo, digamos que o usuário usar a sentença: <code>1); DROP TABLE Produtos;</code> para ao campo <code>valor</code> o comando ficaria: </p>

<pre><code>insert into Produtos (coluna) values(1); DROP TABLE Produtos;
</code></pre>

<p>Ele vai inserir um novo registro cujo o campo <code>coluna</code> será <code>1</code> e logo em seguida ele vai deletar a tabela Produtos.</p>

<p>Como posso melhorar meu código para prevenir essa situação?</p>



In [7]:
re.findall(r'<.*?>', df_portugues.Questão[5])

['<p>',
 '</p>',
 '<pre>',
 '<code>',
 '</code>',
 '</pre>',
 '<p>',
 '<code>',
 '</code>',
 '<code>',
 '</code>',
 '</p>',
 '<pre>',
 '<code>',
 '</code>',
 '</pre>',
 '<p>',
 '<code>',
 '</code>',
 '<code>',
 '</code>',
 '</p>',
 '<p>',
 '</p>']

In [8]:
texto_teste = re.sub(r'<.*?>', 'T---E----S---T---E', questao_portugues)
print(texto_teste)

T---E----S---T---EDesenvolvi uma página em PHP para uso interno da empresa que trabalho e apenas pouquíssimas pessoas a utilizam. Através dessa página é possível fazer algumas consultas, inserções, alterações e remoções de dados de uma tabela em um banco de dados MySQL, porém eu acredito que meu código em PHP não está protegido contra injeção de código SQL, por exemplo:T---E----S---T---E

T---E----S---T---ET---E----S---T---E//----CONSULTA SQL----//
$busca = mysql_query ('insert into Produtos (coluna) values(' . $valor . ')');
T---E----S---T---ET---E----S---T---E

T---E----S---T---ELogo, digamos que o usuário usar a sentença: T---E----S---T---E1); DROP TABLE Produtos;T---E----S---T---E para ao campo T---E----S---T---EvalorT---E----S---T---E o comando ficaria: T---E----S---T---E

T---E----S---T---ET---E----S---T---Einsert into Produtos (coluna) values(1); DROP TABLE Produtos;
T---E----S---T---ET---E----S---T---E

T---E----S---T---EEle vai inserir um novo registro cujo o campo T---E----S-

In [9]:
re.search(r'70', '51154414848784898788578705585545')

<re.Match object; span=(23, 25), match='70'>

In [10]:
regex = re.compile(r'70')

In [11]:
regex.search('51154414848784898788578705585545')

<re.Match object; span=(23, 25), match='70'>

In [12]:
from timeit import timeit

In [13]:
setup = """import re"""
timeit("""re.search(r'70', '51154414848784898788578705585545')""", setup)

1.109464000000001

In [14]:
setup = """import re
regex = re.compile(r'70')""" 
timeit('''regex.search('51154414848784898788578705585545')''', setup)


0.2936829000000003

In [15]:
def remover(textos, regex):
    if type(textos) == str:
        return regex.sub("", textos)
    else:
        return [regex.sub("", texto) for texto in textos]

In [16]:
regex_html = re.compile(r'<.*?>')
questao_sem_tag = remover(questao_espanhol, regex_html)
print(questao_sem_tag)

Siempre he visto que en JavaScript hay:


asignaciones =
comparaciones == y ===


Creo entender que == hace algo parecido a comparar el valor de la variable y el === también compara el tipo (como un equals de java). 

¿Alguien podría confirmarme este punto y extenderlo?. Soy javero y el no tipado de javascript a veces me encanta y otras lo odio.



¿Cuál es la manera correcta en javascript de comparar undefined, null y otros valores por defecto? 

variable == null
variable === null


¿undefined se usa como cadena de texto o como palabra clave? ¿Cual de las siguientes comparaciones es la correcta para un elemento html sin value? (por ejemplo un label sin contenido)

variable == "undefined"
variable === "undefined"
variable == undefined
variable === undefined




In [17]:
def substituir_codigo(textos, regex):
    if type(textos) == str:
        return regex.sub("CODE", textos)
    else:
        return [regex.sub("CODE", texto) for texto in textos]

In [18]:
regex_codigo = re.compile(r'<code>(.|(\n))*?</code>')
questao_sem_codigo = substituir_codigo(questao_ingles, regex_codigo)
print(questao_sem_codigo)

<p>What is the use of the CODE keyword in Python? What does it do?</p>

<p>For example, I'm trying to understand this code<sup><strong>1</strong></sup>:</p>

<pre>CODE</pre>

<p>And this is the caller:</p>

<pre>CODE</pre>

<p>What happens when the method CODE is called?
Is a list returned? A single element? Is it called again? When will subsequent calls stop?</p>

<hr>

<p><sub>
1. This piece of code was written by Jochen Schulz (jrschulz), who made a great Python library for metric spaces. This is the link to the complete source: <a href="http://well-adjusted.de/~jrschulz/mspace/" rel="noreferrer">Module mspace</a>.</sub></p>



In [19]:
questoes_port_sem_code = substituir_codigo(df_portugues.Questão, regex_codigo)
questoes_port_sem_code_tag = remover(questoes_port_sem_code, regex_html)

df_portugues['sem_code_tag'] = questoes_port_sem_code_tag
df_portugues.head()

,Id,Título,Questão,Tags,Pontuação,Visualizações,sem_code_tag
0,2402,Como fazer hash de senhas de forma segura?,"<p>Se eu fizer o <em><a href=""http://pt.wikipe...",<hash><segurança><senhas><criptografia>,350,22367,Se eu fizer o hash de senhas antes de armazená...
1,6441,Qual é a diferença entre INNER JOIN e OUTER JOIN?,<p>Qual é a diferença entre <code>INNER JOIN</...,<sql><join>,276,176953,Qual é a diferença entre CODE e CODE? Podem me...
2,579,Por que não devemos usar funções do tipo mysql_*?,<p>Uma dúvida muito comum é por que devemos pa...,<php><mysql>,226,9761,Uma dúvida muito comum é por que devemos parar...
3,2539,As mensagens de erro devem se desculpar?,<p>É comum encontrar uma mensagem de erro que ...,<aplicação-web><gui><console><ux>,214,5075,É comum encontrar uma mensagem de erro que diz...
4,17501,"Qual é a diferença de API, biblioteca e Framew...",<p>Me parecem termos muito próximos e eventual...,<api><framework><terminologia><biblioteca>,193,54191,Me parecem termos muito próximos e eventualmen...


In [20]:
questoes_ingles_sem_code = substituir_codigo(df_ingles.Questão, regex_codigo)
questoes_ingles_sem_code_tag = remover(questoes_ingles_sem_code, regex_html)

df_ingles['sem_code_tag'] = questoes_ingles_sem_code_tag
df_ingles.head()

,Id,Título,Questão,Tags,Pontuação,Visualizações,sem_code_tag
0,11227809,Why is it faster to process a sorted array tha...,<p>Here is a piece of C++ code that seems very...,<java><c++><performance><optimization><branch-...,23057,1358574,Here is a piece of C++ code that seems very pe...
1,927358,How do I undo the most recent local commits in...,<p>I accidentally committed the wrong files to...,<git><version-control><git-commit><undo>,19640,7906137,I accidentally committed the wrong files to Gi...
2,2003505,How do I delete a Git branch locally and remot...,<p>I want to delete a branch both locally and ...,<git><git-branch><git-remote>,15249,6940906,I want to delete a branch both locally and rem...
3,292357,What is the difference between 'git pull' and ...,<blockquote>\n <p><strong>Moderator Note:</st...,<git><git-pull><git-fetch>,11008,2543052,\n Moderator Note: Given that this question h...
4,477816,What is the correct JSON content type?,"<p>I've been messing around with <a href=""http...",<json><http-headers><content-type>,9701,2478940,I've been messing around with JSON for some ti...


In [21]:
questoes_espanhol_sem_code = substituir_codigo(df_espanhol.Questão, regex_codigo)
questoes_espanhol_sem_code_tag = remover(questoes_espanhol_sem_code, regex_html)

df_espanhol['sem_code_tag'] = questoes_espanhol_sem_code_tag
df_espanhol.head()

,Id,Título,Questão,Tags,Pontuação,Visualizações,sem_code_tag
0,18232,¿Cómo evitar la inyección SQL en PHP?,<p>Las sentencias dinámicas son sentencias SQL...,<php><mysql><sql><seguridad><inyección-sql>,169,38614,Las sentencias dinámicas son sentencias SQL qu...
1,197,¿Por qué mis programas no pueden hacer cálculo...,<p>Unas veces los cálculos funcionan correctam...,<matemáticas><coma-flotante><lenguaje-agnóstico>,141,3860,Unas veces los cálculos funcionan correctament...
2,36,¿Cuál es la diferencia entre un inner y un out...,<p>¿Cuál es la diferencia entre un <code>inner...,<mysql><sql><join>,97,53627,¿Cuál es la diferencia entre un CODE y un CODE...
3,29177,¿Por qué es considerado una mala práctica util...,"<p>La mayoría de nosotros decimos, (muchas vec...",<variables><variables-globales><patrones-de-di...,89,9987,"La mayoría de nosotros decimos, (muchas veces ..."
4,142,Validar un email en JavaScript que acepte todo...,<h3>Pregunta</h3>\n\n<p>¿Cómo validar un e-mai...,<javascript><validación><email><regex>,87,73129,Pregunta\n\n¿Cómo validar un e-mail que acepte...


In [22]:
regex_pontuacao = re.compile(r'[^ \w\s]')
print(remover(questoes_ingles_sem_code_tag[0], regex_pontuacao))

Here is a piece of C code that seems very peculiar For some strange reason sorting the data miraculously makes the code almost six times faster

CODE


Without CODE the code runs in 1154 seconds
With the sorted data the code runs in 193 seconds


Initially I thought this might be just a language or compiler anomaly So I tried it in Java

CODE

With a somewhat similar but less extreme result



My first thought was that sorting brings the data into the cache but then I thought how silly that is because the array was just generated


What is going on
Why is it faster to process a sorted array than an unsorted array
The code is summing up some independent terms and the order should not matter




In [23]:
print(remover(questoes_espanhol_sem_code[0], regex_pontuacao))

pLas sentencias dinámicas son sentencias SQL que se crean como cadenas de texto strings y en las que se insertanconcatenan valores obtenidos de alguna fuente normalmente proveniente del usuario lo que puede hacer que sean vulnerables a inyección SQL si no se sanean las entradas como por ejemplop

preCODEpre

pEso es un ejemplo de una vulnerabilidad grave en la seguridad de una aplicación web o no porque si el usuario introdujese un valor como CODE nos encontraríamos con que la sentencia ejecutada seríap

preCODEpre

pY se eliminaría la tabla Usuarios con todos los datos contenidos en ella p

pstrongCómo puedo evitar que la inyección SQL ocurra en PHPstrongp



In [24]:
def minusculo(textos):
    if type(textos) == str:
        return textos.lower()
    else:
        return [texto.lower() for texto in textos]

In [25]:
print(minusculo(questoes_espanhol_sem_code_tag[0]))

las sentencias dinámicas son sentencias sql que se crean como cadenas de texto (strings) y en las que se insertan/concatenan valores obtenidos de alguna fuente (normalmente proveniente del usuario), lo que puede hacer que sean vulnerables a inyección sql si no se sanean las entradas, como por ejemplo:

code

eso es un ejemplo de una vulnerabilidad grave en la seguridad de una aplicación (web o no) porque si el usuario introdujese un valor como code nos encontraríamos con que la sentencia ejecutada sería:

code

y se eliminaría la tabla usuarios con todos los datos contenidos en ella. 

¿cómo puedo evitar que la inyección sql ocurra en php?



In [26]:
regex_digitos = re.compile(r'\d+')
print(remover('Alura 1234 Caleum 1234566345', regex_digitos))

Alura  Caleum 


In [27]:
regex_espaco = re.compile(r' +')
regex_quebra_linha = re.compile(r'\n')

def substituir_por_espaco(textos, regex):
    if type(textos) == str:
        return regex.sub(" ", textos)
    else:
        return [regex.sub(" ", texto) for texto in textos]

In [28]:
print(substituir_por_espaco('Alura      \n    Caelum        ', regex_espaco))

Alura 
 Caelum 


In [29]:
print(substituir_por_espaco('Alura      \n    Caelum        ', regex_quebra_linha))

Alura           Caelum        


### Tratamento final nas questões
* Eliminando pontuação, 
* Convertendo todas as letras em minusculas,
* Removendo digitos
* Removendo quebra de linha
* Removendo espaços duplicados


In [30]:
questoes_port_sem_pont = remover(df_portugues['sem_code_tag'],
                                 regex_pontuacao)
questoes_port_sem_pont_minus = minusculo(questoes_port_sem_pont)

questoes_port_sem_pont_minus_dig = remover(questoes_port_sem_pont_minus, 
                                           regex_digitos)

questoes_port_sem_quebra_linha = substituir_por_espaco(questoes_port_sem_pont_minus_dig, 
                                                       regex_quebra_linha)

questoes_port_sem_espaco_dupli = substituir_por_espaco(questoes_port_sem_quebra_linha, 
                                                       regex_espaco)

df_portugues['questoes_tratadas'] = questoes_port_sem_espaco_dupli

df_portugues.head()

,Id,Título,Questão,Tags,Pontuação,Visualizações,sem_code_tag,questoes_tratadas
0,2402,Como fazer hash de senhas de forma segura?,"<p>Se eu fizer o <em><a href=""http://pt.wikipe...",<hash><segurança><senhas><criptografia>,350,22367,Se eu fizer o hash de senhas antes de armazená...,se eu fizer o hash de senhas antes de armazená...
1,6441,Qual é a diferença entre INNER JOIN e OUTER JOIN?,<p>Qual é a diferença entre <code>INNER JOIN</...,<sql><join>,276,176953,Qual é a diferença entre CODE e CODE? Podem me...,qual é a diferença entre code e code podem me ...
2,579,Por que não devemos usar funções do tipo mysql_*?,<p>Uma dúvida muito comum é por que devemos pa...,<php><mysql>,226,9761,Uma dúvida muito comum é por que devemos parar...,uma dúvida muito comum é por que devemos parar...
3,2539,As mensagens de erro devem se desculpar?,<p>É comum encontrar uma mensagem de erro que ...,<aplicação-web><gui><console><ux>,214,5075,É comum encontrar uma mensagem de erro que diz...,é comum encontrar uma mensagem de erro que diz...
4,17501,"Qual é a diferença de API, biblioteca e Framew...",<p>Me parecem termos muito próximos e eventual...,<api><framework><terminologia><biblioteca>,193,54191,Me parecem termos muito próximos e eventualmen...,me parecem termos muito próximos e eventualmen...


In [31]:
questoes_espan_sem_pont = remover(df_espanhol['sem_code_tag'],
                                 regex_pontuacao)
questoes_espan_sem_pont_minus = minusculo(questoes_espan_sem_pont)

questoes_espan_sem_pont_minus_dig = remover(questoes_espan_sem_pont_minus, 
                                           regex_digitos)

questoes_espan_sem_quebra_linha = substituir_por_espaco(questoes_espan_sem_pont_minus_dig, 
                                                       regex_quebra_linha)

questoes_espan_sem_espaco_dupli = substituir_por_espaco(questoes_espan_sem_quebra_linha, 
                                                       regex_espaco)

df_espanhol['questoes_tratadas'] = questoes_espan_sem_espaco_dupli

df_espanhol.head()

,Id,Título,Questão,Tags,Pontuação,Visualizações,sem_code_tag,questoes_tratadas
0,18232,¿Cómo evitar la inyección SQL en PHP?,<p>Las sentencias dinámicas son sentencias SQL...,<php><mysql><sql><seguridad><inyección-sql>,169,38614,Las sentencias dinámicas son sentencias SQL qu...,las sentencias dinámicas son sentencias sql qu...
1,197,¿Por qué mis programas no pueden hacer cálculo...,<p>Unas veces los cálculos funcionan correctam...,<matemáticas><coma-flotante><lenguaje-agnóstico>,141,3860,Unas veces los cálculos funcionan correctament...,unas veces los cálculos funcionan correctament...
2,36,¿Cuál es la diferencia entre un inner y un out...,<p>¿Cuál es la diferencia entre un <code>inner...,<mysql><sql><join>,97,53627,¿Cuál es la diferencia entre un CODE y un CODE...,cuál es la diferencia entre un code y un code ...
3,29177,¿Por qué es considerado una mala práctica util...,"<p>La mayoría de nosotros decimos, (muchas vec...",<variables><variables-globales><patrones-de-di...,89,9987,"La mayoría de nosotros decimos, (muchas veces ...",la mayoría de nosotros decimos muchas veces si...
4,142,Validar un email en JavaScript que acepte todo...,<h3>Pregunta</h3>\n\n<p>¿Cómo validar un e-mai...,<javascript><validación><email><regex>,87,73129,Pregunta\n\n¿Cómo validar un e-mail que acepte...,pregunta cómo validar un email que acepte todo...


In [32]:
questoes_ingles_sem_pont = remover(df_ingles['sem_code_tag'],
                                 regex_pontuacao)
questoes_ingles_sem_pont_minus = minusculo(questoes_ingles_sem_pont)

questoes_ingles_sem_pont_minus_dig = remover(questoes_ingles_sem_pont_minus, 
                                           regex_digitos)

questoes_ingles_sem_quebra_linha = substituir_por_espaco(questoes_ingles_sem_pont_minus_dig, 
                                                       regex_quebra_linha)

questoes_ingles_sem_espaco_dupli = substituir_por_espaco(questoes_ingles_sem_quebra_linha, 
                                                       regex_espaco)

df_ingles['questoes_tratadas'] = questoes_ingles_sem_espaco_dupli

df_ingles.head()

,Id,Título,Questão,Tags,Pontuação,Visualizações,sem_code_tag,questoes_tratadas
0,11227809,Why is it faster to process a sorted array tha...,<p>Here is a piece of C++ code that seems very...,<java><c++><performance><optimization><branch-...,23057,1358574,Here is a piece of C++ code that seems very pe...,here is a piece of c code that seems very pecu...
1,927358,How do I undo the most recent local commits in...,<p>I accidentally committed the wrong files to...,<git><version-control><git-commit><undo>,19640,7906137,I accidentally committed the wrong files to Gi...,i accidentally committed the wrong files to gi...
2,2003505,How do I delete a Git branch locally and remot...,<p>I want to delete a branch both locally and ...,<git><git-branch><git-remote>,15249,6940906,I want to delete a branch both locally and rem...,i want to delete a branch both locally and rem...
3,292357,What is the difference between 'git pull' and ...,<blockquote>\n <p><strong>Moderator Note:</st...,<git><git-pull><git-fetch>,11008,2543052,\n Moderator Note: Given that this question h...,moderator note given that this question has a...
4,477816,What is the correct JSON content type?,"<p>I've been messing around with <a href=""http...",<json><http-headers><content-type>,9701,2478940,I've been messing around with JSON for some ti...,ive been messing around with json for some tim...


## Criando o modelo de linguagem com NLTK

Quando utilizamos os bigrams, o primeiro e o último caracter da frase aparece uma única vez, diferentes dos outros caracteres, para resolver isso podemos adicionar um caracter falso para evitar esse tipo de problema, utilizando a função do NLTK - **pad_both_ends**

https://www.nltk.org/api/nltk.lm.html

In [33]:
texto_teste = "Alura"
print(*bigrams(texto_teste))

('A', 'l') ('l', 'u') ('u', 'r') ('r', 'a')


In [34]:
texto_teste = "Alura"
print(*bigrams(pad_both_ends(texto_teste, n=2)))

('<s>', 'A') ('A', 'l') ('l', 'u') ('u', 'r') ('r', 'a') ('a', '</s>')


In [35]:
df_portugues['idioma'] = 'port'
df_espanhol['idioma'] = 'esp'
df_ingles['idioma'] = 'ing'

In [36]:
X_train_port, X_test_port = train_test_split(df_portugues['questoes_tratadas'], 
                                             test_size=0.2, random_state=123)

X_train_ing, X_test_ing = train_test_split(df_ingles['questoes_tratadas'], 
                                             test_size=0.2, random_state=123)

X_train_esp, X_test_esp = train_test_split(df_espanhol['questoes_tratadas'], 
                                             test_size=0.2, random_state=123)

In [37]:
todas_questoes_port = ' '.join(X_train_port)
todas_questoes_port[:1000]

'sou iniciante em php e gostaria de saber se pdophp data objects é a maneira mais segura de se conectar a um banco de dados preciso também de um exemplo de como fazer esta conexão e inserirselecionar dados  por exemplo code estou fazendo um efeito aqui mas não estou entendendo esse sinal de code que tem no framework resultado final \r \r code\r code\r code\r \r \r  qual diferença de linguagem compilada para linguagem interpretada e quais as vantagens de uma para outra  qual a diferença entre code e code ambas são muito parecidas mas em qual caso é melhor usar uma ou a outra gostaria de código de exemplos  um compilador é programado em qual linguagem todos os compiladores de uma linguagem são iguais ou pode existir um melhor que outro  essa pergunta parece ter uma resposta óbvia mas eu confesso que ainda não peguei a ideia sozinho estou trabalhando com orientação a objetos desde principalmente voltado para desenvolvimento web logo que comecei já ouvi falar sobre mvc quando peguei um liv

In [38]:
todas_palavras_port = nltk.tokenize.WhitespaceTokenizer().tokenize(todas_questoes_port)
print(todas_palavras_port[:50])

['sou', 'iniciante', 'em', 'php', 'e', 'gostaria', 'de', 'saber', 'se', 'pdophp', 'data', 'objects', 'é', 'a', 'maneira', 'mais', 'segura', 'de', 'se', 'conectar', 'a', 'um', 'banco', 'de', 'dados', 'preciso', 'também', 'de', 'um', 'exemplo', 'de', 'como', 'fazer', 'esta', 'conexão', 'e', 'inserirselecionar', 'dados', 'por', 'exemplo', 'code', 'estou', 'fazendo', 'um', 'efeito', 'aqui', 'mas', 'não', 'estou', 'entendendo']


In [39]:
X_train_port_bigram, vocab_port = padded_everygram_pipeline(2, todas_palavras_port)

In [40]:
next(next(X_train_port_bigram))

('<s>',)

### 1º Modelo - Maxima verossimilhança

In [41]:
modelo_port = MLE(2)
modelo_port.fit(X_train_port_bigram, vocab_port)

In [42]:
modelo_port.generate(num_words=6)

['<s>', 'd', 'a', 'c', 'e', 's']

In [43]:
from nltk.lm import NgramCounter

modelo_port.counts[['m']].items()

dict_items([('</s>', 2741), ('a', 2052), ('b', 144), ('p', 722), ('o', 854), ('e', 1503), ('u', 227), ('v', 16), ('í', 13), ('i', 290), ('ó', 37), ('â', 11), ('l', 23), ('é', 76), ('ú', 5), ('ã', 4), ('á', 32), ('s', 9), ('m', 25), ('g', 4), ('w', 1), ('_', 1), ('y', 10), ('d', 3)])

In [44]:
texto = 'bom dia'
palavras = nltk.tokenize.WhitespaceTokenizer().tokenize(texto)
palavras_fakechar = [list(pad_both_ends(palavra, n=2)) for palavra in palavras]
palavras_bigramns = [list(bigrams(palavra)) for palavra in palavras_fakechar]
palavras_bigramns

[[('<s>', 'b'), ('b', 'o'), ('o', 'm'), ('m', '</s>')],
 [('<s>', 'd'), ('d', 'i'), ('i', 'a'), ('a', '</s>')]]

In [45]:
print(palavras_bigramns[0])

[('<s>', 'b'), ('b', 'o'), ('o', 'm'), ('m', '</s>')]


In [46]:
print(modelo_port.perplexity(palavras_bigramns[0]))
print(modelo_port.perplexity(palavras_bigramns[1]))


15.298816480210654
8.118235162682486


In [47]:
texto = 'good morning'
palavras = nltk.tokenize.WhitespaceTokenizer().tokenize(texto)
palavras_fakechar = [list(pad_both_ends(palavra, n=2)) for palavra in palavras]
palavras_bigramns = [list(bigrams(palavra)) for palavra in palavras_fakechar]

print(modelo_port.perplexity(palavras_bigramns[0]))
print(modelo_port.perplexity(palavras_bigramns[1]))

47.67050825747126
21.535588522214386


In [48]:
def treinar_modelo_mle(lista_textos):
    todas_questoes = ' '.join(lista_textos)
    todas_palavras = nltk.tokenize.WhitespaceTokenizer().tokenize(todas_questoes)
    X_train_bigram, vocab = padded_everygram_pipeline(2, todas_palavras)
    modelo = MLE(2)
    modelo.fit(X_train_bigram, vocab)
    return modelo
    
    

In [49]:
modelo_port_2 = treinar_modelo_mle(X_train_port)

texto = 'good morning'
palavras = nltk.tokenize.WhitespaceTokenizer().tokenize(texto)
palavras_fakechar = [list(pad_both_ends(palavra, n=2)) for palavra in palavras]
palavras_bigramns = [list(bigrams(palavra)) for palavra in palavras_fakechar]

print(modelo_port_2.perplexity(palavras_bigramns[0]))
print(modelo_port_2.perplexity(palavras_bigramns[1]))

47.67178908687439
21.535806001201696


In [50]:
modelo_ing = treinar_modelo_mle(X_train_ing)
modelo_esp = treinar_modelo_mle(X_train_esp)

In [51]:
print(modelo_ing.perplexity(palavras_bigramns[0]))
print(modelo_ing.perplexity(palavras_bigramns[1]))

17.26133308212415
12.900755877319751


In [52]:
def calcular_perplexidade(modelo, texto):
    perplexidade = 0
    palavras = nltk.tokenize.WhitespaceTokenizer().tokenize(texto)
    palavras_fakechar = [list(pad_both_ends(palavra, n=2)) for palavra in palavras]
    palavras_bigramns = [list(bigrams(palavra)) for palavra in palavras_fakechar]
    
    for palavra in palavras_bigramns:
        perplexidade += modelo.perplexity(palavra)
    
    return perplexidade

In [53]:
print(calcular_perplexidade(modelo_ing, 'good morning'))

30.1620889594439


In [54]:
print(calcular_perplexidade(modelo_port, X_test_port.iloc[0]))

2006.996969376799


In [55]:
print(calcular_perplexidade(modelo_ing, X_test_port.iloc[0]))

inf


### Modelo de laplace
Utiliza-se esse técnica para evitar que a perplexidade seja infinita, esse método consiste em adicionar 1 na probabilidade.

In [58]:
def treinar_modelo_laplace(lista_textos):
    todas_questoes = ' '.join(lista_textos)
    todas_palavras = nltk.tokenize.WhitespaceTokenizer().tokenize(todas_questoes)
    X_train_bigram, vocab = padded_everygram_pipeline(2, todas_palavras)
    modelo = Laplace(2)
    modelo.fit(X_train_bigram, vocab)
    return modelo

In [66]:
modelo_ing_Laplace = treinar_modelo_laplace(X_train_ing)
print(calcular_perplexidade(modelo_ing_Laplace, X_test_port.iloc[0]))

5876.837588345698


In [67]:
modelo_port_Laplace = treinar_modelo_laplace(X_train_port)
print(calcular_perplexidade(modelo_port_Laplace, X_test_port.iloc[0]))

2009.1937946178912


In [68]:
modelo_esp_Laplace = treinar_modelo_laplace(X_train_esp)
print(calcular_perplexidade(modelo_esp_Laplace, X_test_port.iloc[0]))

3488.5698949157722


In [71]:
def atribui_idioma(lista_textos):
    idioma = []
    for texto in lista_textos:
        portugues = calcular_perplexidade(modelo_port_Laplace, texto)
        ingles = calcular_perplexidade(modelo_ing_Laplace, texto)
        espanhol = calcular_perplexidade(modelo_esp_Laplace, texto)
        
        if portugues <= ingles and portugues <= espanhol:
            idioma.append('portugues')
        elif ingles <= espanhol:
            idioma.append('ingles')
        else:
            idioma.append('espanhol')
    return idioma

In [79]:
resultado_portugues = atribui_idioma(X_test_port)
taxa_portugues = resultado_portugues.count('portugues') / len(resultado_portugues)

resultado_ingles = atribui_idioma(X_test_ing)
taxa_ingles = resultado_ingles.count('ingles') / len(resultado_ingles)

resultado_espanhol = atribui_idioma(X_test_esp)
taxa_espanhol = resultado_espanhol.count('espanhol') / len(resultado_espanhol)

In [80]:
print(f'A taxa de acerto para palavras em portugues foi de {taxa_portugues:0.2%}')
print(f'A taxa de acerto para palavras em espanhol foi de {taxa_espanhol:0.2%}')
print(f'A taxa de acerto para palavras em inglês foi de {taxa_ingles:0.2%}')

A taxa de acerto para palavras em portugues foi de 100.00%
A taxa de acerto para palavras em espanhol foi de 97.00%
A taxa de acerto para palavras em inglês foi de 100.00%
